In [1]:
from heat_pump_model_v2 import *
from electric_model import *
from gas_model import *

from libraries import * 
from refrigerant_properties import *
import numpy as np
from utilities.unit_defs import Q_

from cashflow_model import calculate_cash_flow

ModuleNotFoundError: No module named 'plotly'

In [ ]:
hp_test = heat_pump()
hp_test.construct_yaml_input_quantities('heatpump_model_inputs.yml')
hp_test.hot_temperature_desired = Q_(np.array([120]*8760), 'degC')
hp_test.hot_temperature_minimum = Q_(np.array([80]*8760), 'degC')
hp_test.cold_temperature_available = Q_(np.array([60]*8760), 'degC')
hp_test.gas_price_MMBTU = Q_(np.array([4.5] * 8760), 'USD / MMBtu')
hp_test.carnot_efficiency_factor = Q_('0.50')
hp_test.carnot_efficiency_factor_flag = False
hp_test.refrigerant = 'R234a'

hp_test.run_all('hp_test')

In [ ]:
heater_test = electric_heater()
heater_test.construct_yaml_input_quantities('electric_model_inputs.yml')
heater_test.hot_temperature = Q_(np.array([120]*8760), 'degC')
heater_test.cold_temperature = Q_(np.array([60]*8760), 'degC')
heater_test.run_all('electric_test')


In [ ]:
gas_test = gas_heater()
gas_test.construct_yaml_input_quantities('gas_model_inputs.yml')
gas_test.hot_temperature = Q_(np.array([120]*8760), 'degC')
gas_test.cold_temperature = Q_(np.array([60]*8760), 'degC')
gas_test.run_all('gas_test')


In [ ]:
## Converting one component to intput dict



gas_dict = {
    'capital_cost'                  : gas_test.capital_cost.m,
    'year_one_energy_cost'          : gas_test.year_one_energy_costs.m,
    'year_one_fixed_o_and_m'        : gas_test.year_one_fixed_o_and_m.m,
    'year_one_variable_o_and_m'     : gas_test.year_one_variable_o_and_m.m,
    'year_one_cost_of_emissions'    : gas_test.year_one_cost_of_emissions.m
}

elec_dict = {
    'capital_cost'                  : heater_test.capital_cost.m,
    'year_one_energy_cost'          : heater_test.year_one_energy_costs.m,
    'year_one_fixed_o_and_m'        : heater_test.year_one_fixed_o_and_m.m,
    'year_one_variable_o_and_m'     : heater_test.year_one_variable_o_and_m.m,
    'year_one_cost_of_emissions'    : 0
}

calculate_cash_flow(elec_dict, gas_dict, 20, 0.10)